-----------------------------------------------

# Photometry Observations of Didymos Asteroid (Part 2 of 4): 

# Finding Best Aperture Size and Selecting Suitable Comparison Stars 

## By Arushi Nath, Founder: MonitorMyPlanet.com

----------------------------------

#### This Second Notebook will take you through the first steps of differential photometry. Here, the algorithm will be finding the aperture radius that best fits Didymos's size (in pixels) in the images. The chosen aperture radius will then be used to find the total pixel brightness of all stars in the image. Background camera noise will be acounted by creating an outer annulus (ring) around each celestial object and measuring the average background brightness around the star. It will then substract that from the total pixel brightness to get the star's actual pixel brightness. Comparison stars will then be found to account for change in seeing conditions during an observation. Stars are selected as comparison stars only if their brightness is similar to Didymos and if their instrument magnitude correlates to their true magnitude.

------------------------------

### 1. Testing Out Different Aperture Sizes on Didymos

In [ ]:
Total_Brightness_ast = []
Brightness_ast = []
ast_single = []

start = 4
end = 14
Per_p = 16
X_axis = []

for img in tqdm(range(len(images))): # Loop through each image
    centroids_ast = np.array(find_complete_star(images[img], didymos_x, didymos_y))
    X_axis = []
    for tp in tqdm(range((end-start)*Per_p)):
        t = start
        ap=tp/Per_p + start
        X_axis.append(ap)

        NEW_IMAGE = images[img]

        coord_edit_ast = []
        coord_edit_ast.append(centroids_ast[img])

        data = NEW_IMAGE
        norm_ast = simple_norm(data, 'sqrt', percent=99)

        positions_ast = [x for x in coord_edit_ast if 'nan' not in str(x)]
       

        aperture_ast = CircularAperture(positions_ast, r=ap)
        annulus_aperture_ast = CircularAnnulus(positions_ast, r_in=ap+8, r_out=ap+12)
        
        ap_patches = aperture_ast.plot(color='white', lw=2)
        ann_patches = annulus_aperture_ast.plot(color='red', lw=2)
        
        aperstats = ApertureStats(data, annulus_aperture_ast)
        bkg_mean = aperstats.mean

        phot_table = aperture_photometry(data, aperture_ast)
        for col in phot_table.colnames:
            phot_table[col].info.format = '%.8g'  # for consistent table output

        aperture_area = aperture_ast.area_overlap(data)
        total_bkg = bkg_mean * aperture_area
    
        phot_bkgsub = phot_table['aperture_sum'] - total_bkg
    
        phot_table['total_bkg'] = total_bkg
        phot_table['aperture_sum_bkgsub'] = phot_bkgsub
        for col in phot_table.colnames:
            phot_table[col].info.format = '%.8g' 
     
        for i in phot_table:
            ast_single.append(i[5])
   
        Brightness_ast.append(ast_single)
        ast_single = []
      
    Total_Brightness_ast.append([Brightness_ast])
    Brightness_ast = []


In [ ]:
### Re-aranging the Values by Image instead of Aperture Size ###
total_ast= []
for n in range(len(Total_Brightness_ast[0][0][0])):
    semi_star = []
    for s in range(len(Total_Brightness_ast)):

        for i in Total_Brightness_ast[s][0]:
    total_ast.append(semi_ast) 

In [ ]:
### Plotting the Pixel Brightness of Didymos With Different Aperture Sizes in each Image ###
for i in range(len(total_ast)):
    for h in range(len(total_ast[i])):
        plt.plot(X_axis,total_ast[i][h])
    plt.title("Change in Sum of the Brightness of Pixels Captured with Varying Aperture Size of Star Across all Images in Observation")
    plt.xlabel("Aperture Size (Pixels)")
    plt.ylabel("Total Brightness")
    plt.show()

### 2. Finding Which Aperture Size Worked the Best with Didymos

In [ ]:
### Functions used to Find at what Aperture the above Plots have a Slope of 1 ###

def derivative(x):    
    params = list(model5)[:-1]
   
    a = 5*params[0]
    b = 4*params[1]
    c = 3*params[2]
    d = 2*params[3]
    e = 1*params[4]
    return a*(x**4) + b*(x**3) + c*(x**2) + d*(x**1) + e

def find_nearest(array, value):
    array = np.asarray(array)[0:int(len(array))]
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
### This uses the Pixel Brightness of Didymos With Different Aperture Sizes Graph and Gind for each Image their 5th Order Curve Fit ###
### It then uses the Functions above to Find the Ideal Aperture radius for each Image ###

apertures = []

for i in tqdm(range(len(total_ast))):
    for h in tqdm(range(len(total_ast[i]))):
        x = X_axis
        y = total_ast[i][h]
        df = pd.DataFrame({'x': x,
                           'y': y})
        
        model5 = np.poly1d(np.polyfit(df.x, df.y, 5)) #fit polynomial models up to degree 5

        polyline = np.linspace(start, end, 500000)
        plt.scatter(df.x, df.y)

        plt.plot(polyline, model5(polyline), color='orange')
        plt.show()
        
        slopes = [derivative(i) for i in polyline]
        slopes = np.array(slopes)
        
        plt.plot(polyline, slopes)
        
        sh_a = find_nearest(slopes, 1)
    
        plt.scatter(polyline[sh_a], slopes[sh_a], color='r')
        plt.show()
        
        plt.plot(polyline, model5(polyline), color='orange')
        plt.scatter(polyline[sh_a], model5(polyline)[sh_a], color='r')
        plt.show()
        apertures.append(polyline[sh_a])

In [ ]:
#### Creating a Bar Graph to Show the Best Aperture Radius (Pixels) found for each image ###
z = []
for i in range(len(apertures)):
    z.append(i)
    
plt.figure(figsize=(5, 4))

plt.bar(z,apertures)
plt.title("Best Aperture Sizes Determined for each Image in Observation")
plt.xlabel("Image Number")
plt.ylabel("Aperture Size (Pixels)")

In [ ]:
### Finding the Average Aperture radius for all images ###
### The average is the Aperture radius that will be used for all Photometry ###
ast_aperture = np.mean(apertures)

### 3. Finding the Total Pixel Brightness of all Stars in the Image using the Aperture Radius Found

In [ ]:
### For each Image the Code follows the Following steps: ###
### 1. Take the Centroided Positions of all Stars in the Image (Covered in more detail in Part 1) ###
### 2. For each Star, Create a Circle around its Center with the Aperture Radius we have just Determined ###
### 3. Add the Pixel Brightness of all Pixels in the Circle. Lets call the Resultant Number T###
### 4. Create a Ring Around the Star, and find the Average Pixel Value inside that Ring. This will be the Average Noise per Pixel ###
### 5. Multiply the Result of Step 4 by the Amount of Pixels Inside the Circle Created on Step 2. Lets call the resultant Number N ###
### 6. T-N Will give you the Total Pixel Brightness of the Star ###
### 7. Repeat Steps 2 to 6 for all Stars in the Image ###
### 8. The Final Output will be the Total Brightness of All Stars, and how it Changes in each Image ###

Total_Brightness = []
Brightness = []
star_magnitude = []
centroids = np.array(find_complete_star(images[0], star_x,star_y))
for img in tqdm(range(len(images))):

        

    NEW_IMAGE = images[img]#[390:440, 190:350]

    coord_edit = []
    star_magnitude = []
    
    for i in centroids:
        coord_edit.append(i)
    positions = [] 
    data = NEW_IMAGE
    norm = simple_norm(data, 'sqrt', percent=99)
    for x in range(len(coord_edit)):
        if 'nan' not in str(coord_edit[x]):
            positions.append(coord_edit[x])
            star_magnitude.append(rmag_f[x])
            
     
    aperture = CircularAperture(positions, r=ast_aperture)
    annulus_aperture = CircularAnnulus(positions, r_in=ast_aperture+8, r_out=ast_aperture+12)

    plt.imshow(data,cmap='gray', vmin=image_data.mean()-0.2*image_data.std(), vmax=image_data.mean()+0.2*image_data.std(), interpolation='nearest')
    plt.show()

    ap_patches = aperture.plot(color='white', lw=2)
    ann_patches = annulus_aperture.plot(color='red', lw=2)


        
    aperstats = ApertureStats(data, annulus_aperture)
    bkg_mean = aperstats.mean
     

       
    phot_table = aperture_photometry(data, aperture)
    for col in phot_table.colnames:
        phot_table[col].info.format = '%.8g'  # for consistent table output


    aperture_area = aperture.area_overlap(data)


    total_bkg = bkg_mean * aperture_area
    

        
    
    phot_bkgsub = phot_table['aperture_sum'] - total_bkg
    
    phot_table['total_bkg'] = total_bkg
    phot_table['aperture_sum_bkgsub'] = phot_bkgsub
    for col in phot_table.colnames:
        phot_table[col].info.format = '%.8g'  # for consistent table output
  
    
     
    for i in phot_table:
        Brightness.append(i[5])
       
    Total_Brightness.append(Brightness)
    Brightness = []


In [ ]:
### Re-aranging the Pixel Brightness Values by Star Instead of by Image ###

star_mag = []
stars = []
Star_X = []
for i in range(len(Total_Brightness[0])):
    for s in range(len(Total_Brightness)):
        stars.append(Total_Brightness[s][i])
    star_mag.append(stars)
    stars = []

for n in range(len(star_mag[0])):
    Star_X.append(n+1)

In [ ]:
### Plotting the Brightness of Each Star across all Images ###

for l in tqdm(range(len(star_mag))):
    plt.figure(figsize=(15, 15))
    plt.scatter(Star_X, preprocessing.normalize([star_mag[l]]), s=15, color = 'red')
    plt.title("Star RAW CCD Brightness") #title
    plt.xlabel("Time") #x label
    plt.ylabel("Total Pixel Brightness") #y label
   # plt.yscale('log')
    plt.ylim([0, 1])
    plt.show()

## 4. Finding Suitable Comparison Stars
### a)  Eliminating Those Whose Instrument Magnitude Stray from the Best Fit Line
### b) Eliminating Those Whose Magnitude is not Similar to Didymos

In [ ]:
### Finding the Instrument Magnitude of each Star using the Average of their Total Pixel Brightness Across all Images ###

IM_S = []
comp_magnitude = []
for l in range(len(star_mag)):
    a = stats.gmean(star_mag[l])
    IM_S.append(-2.5*np.log(a/60))
   

In [ ]:
### Plotting Instrument Magnitude (Just Calculated) and True Magnitude (from the GAIA Database) for all Stars ###
### All points should Follow a Linear Line with Slope of Approximately 1 ###

plt.scatter(IM_S, star_magnitude, s = 1)

In [ ]:
### Removing Empty "nan" Values from Instrument Magnitude List ###
IM_S2 = []
catalog_mag = []
for i in range(len(IM_S)):
    if str(IM_S[i]) != str('nan'):
        IM_S2.append(IM_S[i])
        catalog_mag.append(star_magnitude[i])

In [ ]:
### Finding Slope of Instrument Magnitude vs True Magnitude Scatter Plot ###
comp_slope = (model5(polyline[-1])-model5(polyline[0]))/(max(df.x) - min(df.x))
print(comp_slope) # Should be Close to 1

In [ ]:
### Finding all Stars whose Brightness is Very Similar to Didymos ###
comp_stars_catalog = []
nums = []
star_IM_comps = []
star_magnitude_comps = []
for i in range(len(star_mag)):
    if np.mean(star_magnitude[i]) > float(ast_catalog_mag[0]) - 0.5 and np.mean(star_magnitude[i]) < float(ast_catalog_mag[0]) + 0.5:
        comp_stars_catalog.append(star_magnitude[i])
        nums.append(i)
        star_IM_comps.append(star_mag[i])

In [ ]:
### Plotting Instrument Magnitude over Time of all Stars whose Brightness is Very Similar to Didymos ###

for l in tqdm(range(len(nums))):
    plt.figure(figsize=(15, 15))
    plt.scatter(Star_X, preprocessing.normalize([star_IM_comps[l]]), s=15, color = 'red')
    title = "Comp Star Instrument Brightness: ", comp_stars_catalog[l]
    plt.title(title) #title
    plt.xlabel("Time") #x label
    plt.ylabel("Raw Pixel Brightness") #y label
   # plt.yscale('log')
    plt.ylim([0, 1])
    plt.show()

In [ ]:
### Plotting Instrument Magnitude and True Magnitude for all Stars with Magnitude between 8 and 18 ###

df = pd.DataFrame({'x': IM_S2,
                   'y': catalog_mag})
plt.scatter(df.x, df.y)

In [ ]:
### Finding Best Linear Fit to above Scatter Plot ###

test = []
test2 = []
for i in range(len(catalog_mag)):
    if catalog_mag[i] < 18 and catalog_mag[i] > 8:
        test.append(IM_S2[i])
        test2.append(catalog_mag[i])

In [ ]:
### Plotting Best Linear Fit to above Scatter Plot ###

df = pd.DataFrame({'x': test,
                   'y': test2})
        
model5 = np.poly1d(np.polyfit(df.x, df.y, 1))

polyline = np.linspace(min(df.x), max(df.x), 100)
plt.scatter(df.x, df.y)

plt.plot(polyline, model5(polyline), color='orange')
plt.show()

In [ ]:
### Finding the Instrument Magnitude of each Star whose Brightness is Similar to Didymos ###
IM_Sc = []
comp_magnitude = []
for l in range(len(star_IM_comps)):
    a = stats.gmean(star_IM_comps[l])
    IM_Sc.append(-2.5*np.log(a/60))
    

In [ ]:
### Plotting Scatter Plot and Best Linear Fit ###

polyline = np.linspace(min(IM_Sc), max(IM_Sc), 100)
plt.scatter(IM_Sc, comp_stars_catalog)
plt.plot(polyline, model5(polyline), color='orange')
title = "Line Slope = ", comp_slope
plt.title(title)
plt.show()

In [ ]:
### Removing all Stars whose Points Stray from the Linear Fit Line ###
IM_Scomp = []
comp_stars_catalog2 = []
star_IM_comps2 = []
for i in range(len(IM_Sc)):
    d = abs(model5(IM_Sc[i]) - comp_stars_catalog[i])
    if d <= 0.8:
        IM_Scomp.append(IM_Sc[i])
        comp_stars_catalog2.append(comp_stars_catalog[i])
        star_IM_comps2.append(star_IM_comps[i])

In [ ]:
### Plotting Scatter Plot and Best Linear Fit of Remaining Stars ###
### These are all the Comparison Stars Selected for the duration of the Observation ###

polyline = np.linspace(min(IM_Scomp), max(IM_Scomp), 100)
plt.scatter(IM_Scomp, comp_stars_catalog2)
plt.plot(polyline, model5(polyline), color='orange')
title = "Line Slope = ", comp_slope
plt.title(title)
plt.show()

In [ ]:
### Plotting Normalized Brightness over Time for all Comparison Stars ###
for l in tqdm(range(len(comp_stars_catalog2))):
    plt.figure(figsize=(15, 15))
    plt.scatter(Star_X, preprocessing.normalize([star_IM_comps2[l]]), s=15, color = 'red')
    title = "Comp Star Normalized Brightness: ", comp_stars_catalog2[l]
    plt.title(title) #title
    plt.xlabel("Time") #x label
    plt.ylabel("Raw ADU") #y label
   # plt.yscale('log')
    plt.ylim([0, 1])
    plt.show()

In [ ]:
### Re-Computing Instrument Magnitude for the Comparison Stars Selected ###
IM = []
comp_instrument_mag = []
for l in range(len(star_IM_comps2)):
    for i in range(len(star_IM_comps2[l])):
        IM.append(-2.5*np.log(star_IM_comps2[l][i]/60))
        
    comp_instrument_mag.append(IM)
    IM = []

In [ ]:
### Plotting Instrument Magnitude over Time for all Comparison Stars ###
for l in tqdm(range(len(comp_instrument_mag))):
    plt.figure(figsize=(15, 15))
    plt.scatter(Star_X, [comp_instrument_mag[l]], s=15, color = 'red')
    title = "Comp Star Instrument Brightness: ", comp_stars_catalog2[l]
    plt.title(title) #title
    plt.xlabel("Time") #x label
    plt.ylabel("Raw ADU") #y label
   # plt.yscale('log')
    plt.ylim([-22, -18])
    plt.show()